In [1]:
#download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-12-30 07:22:48--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2023-12-30 07:22:48 (21.3 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
text = open('input.txt', 'r', encoding='utf-8').read()

In [3]:
print('length of the dataset in characters:', len(text))

length of the dataset in characters: 1115394


In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(chars)
print(vocab_size)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65


In [6]:
stoi = {s:i for i, s in enumerate(chars)}
itos = {i:s for s, i in stoi.items()}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [7]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [8]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
for i in range(block_size):
  context = x[:i+1]
  target = y[i]
  print(f'input {context} and target {target}')

input tensor([18]) and target 47
input tensor([18, 47]) and target 56
input tensor([18, 47, 56]) and target 57
input tensor([18, 47, 56, 57]) and target 58
input tensor([18, 47, 56, 57, 58]) and target 1
input tensor([18, 47, 56, 57, 58,  1]) and target 15
input tensor([18, 47, 56, 57, 58,  1, 15]) and target 47
input tensor([18, 47, 56, 57, 58,  1, 15, 47]) and target 58


In [10]:
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
n_embd = 384
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
eval_iters = 200

cuda


In [12]:
torch.manual_seed(1337)

block_size = 256 # context length
batch_size = 64 #how many independent sequences we process in parallel

def get_batch(split):
  data = train_data if split == 'train' else val_data
  idx = torch.randint(len(data) - block_size, (batch_size, ))
  X = torch.stack([data[i:i+block_size] for i in idx])
  y = torch.stack([data[i+1:i+block_size+1] for i in idx])
  X, y = X.to(device), y.to(device)
  return X, y



Xb, yb = get_batch('train')
print(Xb.shape)
print(Xb)
print(yb.shape)
print(yb)

torch.Size([64, 256])
tensor([[ 0, 26, 53,  ..., 56, 43, 47],
        [60, 43, 56,  ..., 56,  1, 41],
        [26, 21, 33,  ..., 26, 21, 13],
        ...,
        [ 5, 57,  1,  ...,  1, 35, 47],
        [56, 53, 53,  ..., 59, 50, 42],
        [42, 47, 56,  ..., 39, 56,  1]], device='cuda:0')
torch.Size([64, 256])
tensor([[26, 53, 58,  ..., 43, 47, 45],
        [43, 56,  1,  ...,  1, 41, 53],
        [21, 33, 31,  ..., 21, 13, 10],
        ...,
        [57,  1, 52,  ..., 35, 47, 50],
        [53, 53, 58,  ..., 50, 42,  1],
        [47, 56, 43,  ..., 56,  1, 51]], device='cuda:0')


In [13]:
eval_interval = 300

In [14]:
import torch.nn as nn
import torch.nn.functional as F

In [15]:
num_heads = 6
n_layer = 6
dropout = 0.2

In [16]:
class Head(nn.Module):
  ''' One head of self attention '''

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B,T,C = x.shape
    k = self.key(x) # (B, T, C)
    q = self.query(x) # (B, T, C)
    #compute attention scores ("affinities")
    wei = q @ k.transpose(-2, -1) * C**-0.5 # (B,T, head_size) @ (B, head_size, T) ---------> (B, T, T)
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
    wei = F.softmax(wei, dim=-1)
    wei = self.dropout(wei)
    #performed the weighed aggregation of the values
    v = self.value(x)
    out = wei @ v
    return out

In [17]:
class MultiHeadAttention(nn.Module):
  """ multiple heads of self-attention in parallel """

  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embd, n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.dropout(self.proj(out))
    return out

In [18]:
class FeedForwad(nn.Module):
  """ a simple linear layer followed by a non-linearity """

  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embd, 4 * n_embd),
        nn.ReLU(),
        nn.Linear(4 * n_embd, n_embd),
        nn.Dropout(dropout)
    )

  def forward(self, x):
    return self.net(x)


In [19]:
class Block(nn.Module):
  """ Transformer block: communication followed by computation """

  def __init__(self, n_embd, num_heads):
    #n_embd: embedding dimension, n_heads the number of heads we'd like
    super().__init__()
    head_size = n_embd // num_heads
    self.sa = MultiHeadAttention(num_heads, head_size)
    self.ffwd = FeedForwad(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x

In [20]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.blocks = nn.Sequential(*[Block(n_embd, num_heads) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embd)
    #self.sa_heads = MultiHeadAttention(4, n_embd//4)
    #self.ffwd = FeedForwad(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None):
    # interpret logits - as the scores for the next character in the sequence
    B, T = idx.shape
    # idx and targets are both (B,T) tensor of integers
    tok_emb = self.token_embedding_table(idx) # (B, T, C)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device)) #(T, C)
    x = tok_emb + pos_emb # (B, T, C)
    x = self.blocks(x) # (B, T, C)
    logits = self.lm_head(x) # (B, T, vocab_size)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
    return logits, loss

  def generate(self, idx, max_new_tokens):
    #idx is. (B, T) tensor of indices in the current context
    for i in range(max_new_tokens):
      #crop idx to the last block_size tokens
      idx_cond = idx[:, -block_size:]
      #get predictions
      logits, loss = self(idx_cond)
      #focus only on the last time step
      logits = logits[:, -1, :] # (B, C)
      probs = F.softmax(logits, dim=-1) #(B, C)
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      #append sampled index to the running sequence
      idx = torch.cat((idx, idx_next), dim=1) #(B, T+1)
    return idx


model = BigramLanguageModel()
m = model.to(device)
logits, loss = m(Xb, yb)
print(logits.shape)
print(loss)

torch.Size([16384, 65])
tensor(4.4857, device='cuda:0', grad_fn=<NllLossBackward0>)


In [21]:
print(decode(m.generate(idx=torch.zeros((1, 1), dtype=torch.long, device=device), max_new_tokens=500)[0].tolist()))


?pvLRCUZ maQJr?F,E.HfmNDhHKHQEuAsA3WW bE,CVgX
nO,reYG AQj!
..3xfs?r,yLS,SRinHPdGdHSw;kmVSM!WwYCzlLX'koDcnaWqbPEdqZO;mwHKYSqnQcHkecINe-DRJp
VrUso&a?bBFTnYi
?WivAP!hy
tEnaI3AolfVUB.hGHZS:i;;W,S.:xo-oJJ3'u
CT?xE;gcL$
AiZO
sIhiSMkt m
YOtgCD!CaHY!oxZIVfxuEDaBZIxizLcAQcJFLhnUDziaB$qU3 NCD vIdNLRBHSUDYB$m3WW$ HOQci-iZHXRCnZ?rDfZ!-K;DNDvjbIinDvFKDeDPK$tXto!jJQ,SkPHDGc'NcFqEQ$htDDIjf$Db'!ucqF3JDBkgXCmjpeA! SUPRHncek DlnQDnqqn;cF!cDIT
.CAG&LDDf!cWDnRS$OQzhyFDvH$KZvanDKKiFSF'yzDrTjNXW!DazKJ$hBDHXMZZtTnrBCD


In [22]:
optimizer = torch.optim.AdamW(m.parameters(), lr=3e-4)

In [23]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, y = get_batch(split)
      logits, loss = model(X, y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [24]:
batch_size = 32

for iter in range(5000):

  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  Xb, yb = get_batch('train')

  #evaluate loss
  logits, loss = m(Xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()



step 0: train loss 4.4749, val loss 4.4704
step 300: train loss 2.4228, val loss 2.4520
step 600: train loss 2.0784, val loss 2.1491
step 900: train loss 1.8290, val loss 1.9541
step 1200: train loss 1.6702, val loss 1.8331
step 1500: train loss 1.5815, val loss 1.7658
step 1800: train loss 1.5123, val loss 1.7020
step 2100: train loss 1.4527, val loss 1.6598
step 2400: train loss 1.4098, val loss 1.6218
step 2700: train loss 1.3756, val loss 1.5985
step 3000: train loss 1.3458, val loss 1.5705
step 3300: train loss 1.3198, val loss 1.5674
step 3600: train loss 1.2940, val loss 1.5367
step 3900: train loss 1.2754, val loss 1.5360
step 4200: train loss 1.2474, val loss 1.5149
step 4500: train loss 1.2308, val loss 1.5065
step 4800: train loss 1.2183, val loss 1.5120


In [25]:
print(decode(m.generate(idx=torch.zeros((1, 1), dtype=torch.long, device=device), max_new_tokens=500)[0].tolist()))


To friar, and beck at his shall ashe found.

DUKE OF YORK:
The bushet aunt home of coming his crown,
And break with his prince; for, then my dear
My soul, dogs: is the friend; in't, 'tis but of the
secureth her bushed hated of the world,
More our perish? Go girl, what of fear. When you so?

QUEEN:
She's we understry, of bles, the Duke of QYORK:
Then grace hath her content.

QUEEN ELIZABETH:
'Tis a third's here boy.

GLOUCESTER:
Clarenceiff crown back, the rasher stoke?

GRRY BUCY:
It may saying 


In [11]:
#self-attention version
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)
B,T,C = 4,8,16
x = torch.randn(B,T,C)
print(x.shape)

#let's see a single Head perform self-attention
head_size = 16 # hyperparameter
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
v = value(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ----> (B, T, T)

tril = torch.tril(torch.ones((T,T)))
#wei = torch.zeros((T, T))
wei = wei.masked_fill(tril[:T, :T] == 0, float('-inf'))
print(wei[1])
wei = F.softmax(wei, dim=-1)
print(wei[0])
out = wei @ v

out.shape

torch.Size([4, 8, 16])
tensor([[-0.3415,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 1.2285, -0.8629,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 1.7037,  3.1838,  1.9836,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 1.7477, -3.3035,  1.5902, -1.0486,    -inf,    -inf,    -inf,    -inf],
        [-5.5747,  1.0614,  2.5805,  3.0092,  0.5576,    -inf,    -inf,    -inf],
        [ 2.4124,  0.8532,  0.8213, -1.4395, -0.0863, -0.2398,    -inf,    -inf],
        [-1.1077,  1.6303,  1.8502, -0.0631, -0.1566,  0.1737, -0.7321,    -inf],
        [ 1.5674,  0.0998, -0.1855,  0.9613,  0.2184, -0.3162, -1.1050, -2.0756]],
       grad_fn=<SelectBackward0>)
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6619, 0.3381, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4880, 0.2739, 0.2382, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1467, 0.6208, 0.0226, 0.2100, 0.0000, 0.0000, 0.0000,

torch.Size([4, 8, 16])

Notes:

* Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.

* There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.

* Each example across batch dimension is of course processed completely independently and never "talk" to each other

* In an "encoder" attention block just delete the single line that does masking with tril, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.

* "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)

* "Scaled" attention additional divides wei by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [1]:
class Head(nn.Module):
  ''' One head of self attention '''

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

  def forward(self, x):
    B,T,C = x.shape
    k = self.key(x) # (B, T, head_size)
    q = self.query(x) # (B, T, head_size)
    #compute attention scores ("affinities")
    wei = q @ k.transpose(-2, -1) * head_size**-0.5 # (B,T, head_size) @ (B, head_size, T) ---------> (B, T, T)
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
    wei = F.softmax(wei, dim=-1)
    #performed the weighed aggregation of the values
    v = self.value(x)
    out = wei @ v
    return out

NameError: ignored